In [ ]:
from datetime import datetime, timedelta

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, udf
from pyspark.sql.types import IntegerType, LongType

In [ ]:
spark = SparkSession\
    .builder\
    .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11")\
    .enableHiveSupport()\
    .getOrCreate()

input_data = "s3a://paulo-dend/data"
output_data = "s3a://paulo-dend/tables"

In [ ]:
@udf(IntegerType())
def get_date_part(date_delta, key):
    date = datetime(year=1960, month=1, day=1) + timedelta(days=int(date_delta))
    parts = {
        "day": date.day,
        "weekday": date.isoweekday(),
        "month": date.month,
        "year": date.year
    }
    return parts[key]


df = spark \
    .read \
    .format("com.github.saurfang.sas.spark") \
    .load(f"{input_data}/i94_apr16_sub.sas7bdat")

date_parts = ["day", "weekday", "month", "year"]
for part in date_parts:
    df = df.withColumn(part, get_date_part("arrdate", lit(part)))

df.createOrReplaceTempView("arrivals")

time_table = spark.sql("""
    SELECT DISTINCT arrdate, day, weekday, month, year FROM arrivals
""")
time_table \
    .write \
    .partitionBy("year", "month") \
    .mode("overwrite") \
    .parquet(f"{output_data}/time.parquet")

people_table = df.select(
    df.admnum.cast(LongType()).alias("admission_number"),
    df.biryear.cast(IntegerType()).alias("birth_year"),
    df.gender,
    df.i94addr.alias("address_state")
)
people_table \
    .write \
    .partitionBy("gender", "birth_year") \
    .mode("overwrite") \
    .parquet(f"{output_data}/people.parquet")

visas_table = spark.sql("""
    SELECT DISTINCT visatype AS type,
    CASE
        WHEN i94visa = 1.0 THEN 'business'
        WHEN i94visa = 2.0 THEN 'leisure'
        WHEN i94visa = 3.0 THEN 'student'
    END AS purpose
    FROM arrivals
""")
visas_table \
    .write \
    .mode("overwrite") \
    .parquet(f"{output_data}/visas.parquet")

arrivals_table = df.select(
    df.cicid.cast(LongType()).alias("id"),
    df.admnum.cast(LongType()).alias("person_id"),
    df.visatype.alias("visa"),
    df.i94res.cast(IntegerType()).alias("country_of_origin"),
    df.i94port.alias("port_of_entry"),
    df.arrdate.cast(LongType()).alias("date"),
    df.airline,
    df.fltno.alias("flight_number"),
    df.dtaddto.alias("allowed_until_date"),
    df.visapost.alias("visa_issuer")
)
arrivals_table \
    .write \
    .partitionBy("date") \
    .mode("overwrite") \
    .parquet(f"{output_data}/arrivals.parquet")

In [ ]:
US_STATES = (
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA",
    "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY",
    "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX",
    "UT", "VT", "VA", "WA", "WV", "WI", "WY"
)

df = spark.read.csv(f"{input_data}/i94_ports_all.csv", header=True).dropna()
df.createOrReplaceTempView("ports_raw")

ports_of_entry_table = spark.sql(f"""
    SELECT DISTINCT i94_port_code AS code, i94_port_name AS name, i94_port_state AS state
    FROM ports_raw
    WHERE i94_port_state IN {US_STATES}
""")
ports_of_entry_table \
    .write \
    .mode("overwrite") \
    .parquet(f"{output_data}/ports_of_entry.parquet")

In [ ]:
df = spark.read.csv(f"{input_data}/i94_countries_all.csv", header=True).dropna()

df = df.where(
    ~(df.i94_country_name.startswith('No Country Code')
      | df.i94_country_name.startswith('Collapsed'))
)
df = df.withColumn("country_code_as_int", df.i94_country_code.cast(IntegerType()))

remove_prefix_udf = udf(lambda value, prefix: value.replace(prefix, ''))
df = df.withColumn(
    "country_name_without_prefix", remove_prefix_udf("i94_country_name", lit("INVALID: "))
)

countries_table = df.select(
    df.country_code_as_int.alias("i94_code"),
    df.country_name_without_prefix.alias("name")
)
countries_table \
    .write \
    .mode("overwrite") \
    .parquet(f"{output_data}/countries.parquet")

In [ ]:
ports = spark.read.parquet(f"{output_data}/ports_of_entry.parquet")
print(ports.count())
ports.where(ports.code == "NYC").show()

In [ ]:
countries = spark.read.parquet(f"{output_data}/countries.parquet")
print(countries.count())
countries.where(countries.name == "BRAZIL").show()

In [ ]:
arrivals = spark.read.parquet("out/arrivals.parquet")
print(arrivals.count())
arrivals.show(10, truncate=False)